# Inference for 🦜BirdCLEF with Random data

**Clarification about the submission format: https://www.kaggle.com/c/birdclef-2022/discussion/308009**

In [ ]:
!ls -l /kaggle/input/birdclef-2022

PATH_DATASET = "/kaggle/input/birdclef-2022"

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# df_test = pd.read_csv(os.path.join(PATH_DATASET, "test.csv")).set_index("row_id")
# display(df_test)

# Fake audio to dataset

The true conversion is in **https://www.kaggle.com/jirkaborovec/birdclef-convert-spectrograms-noise-reduce**

In [ ]:
import os
import glob
import torch
import torchaudio
from math import ceil
from tqdm.auto import tqdm
from functools import partial
from joblib import Parallel, delayed


def convert_fake(fn, path_in, frame_length: int = 5) -> list:
    path_audio = os.path.join(path_in, fn)
    waveform, sample_rate = torchaudio.load(path_audio)
    nb = int(frame_length * sample_rate)
    counts = ceil(waveform.size()[-1] / nb)
    records = [{
        "end_time": (i + 1) * frame_length,
        "file_id": os.path.splitext(fn)[0],
    } for i in range(counts)]
    return records

_convert_fake = partial(convert_fake, path_in=os.path.join(PATH_DATASET, "test_soundscapes"))

In [ ]:
soundscapes = glob.glob(os.path.join(PATH_DATASET, "test_soundscapes", "*.ogg"))
soundscapes = list(map(os.path.basename, soundscapes))
converted = []
for batch in Parallel(n_jobs=3)(delayed(_convert_fake)(fn) for fn in tqdm(soundscapes)):
    converted += batch
# _= list(map(_convert_and_export, tqdm(train_meta["filename"])))

df_converted = pd.DataFrame(converted)
display(df_converted.head())

# fake Inference with magic

In [ ]:
import json

with open(os.path.join(PATH_DATASET, "scored_birds.json")) as fp:
    scored_birds = json.load(fp)

print(scored_birds)

In [ ]:
import numpy as np
from pprint import pprint

predictions = []
for _ in tqdm(range(len(df_converted))):
    # lbs = [torch.argmax(p["preds"].float()).item() for p in preds]
    predictions += [np.random.random(len(scored_birds))]

pprint(predictions[:2])

# Format submission

Untill this is resolved https://www.kaggle.com/c/birdclef-2022/discussion/309001

In [ ]:
submission = []
for i, row in tqdm(df_converted.iterrows(), total=len(df_converted)):
    preds = dict(zip(scored_birds, predictions[i]))
    for bird in scored_birds:
        submission.append({
            "row_id": f"{row['file_id']}_{bird}_{row['end_time']}",
            "target": preds[bird] > 0.3,
            # "target": True,
            # "target": False,
        })

In [ ]:
df_submission = pd.DataFrame(submission).set_index("row_id")
df_submission.to_csv("submission.csv")

! head submission.csv